# Hybrid Encryption with PostgreSQL pgcrypto

โน้ตบุ๊กนี้สาธิตการใช้ **Hybrid Encryption** เพื่อเข้ารหัสข้อมูลหนึ่งชุดให้ถอดรหัสได้โดยหลายผู้ใช้

## หลักการ Hybrid Encryption
```
┌───────────────────────────────┐
│ Encrypted data (AES)          │ ← เข้ารหัสด้วย session key  
│ +                             │  
│ Encrypted session key A (RSA) │ ← สำหรับ user A  
│ Encrypted session key B (RSA) │ ← สำหรับ user B  
│ ...                           │  
└───────────────────────────────┘
```

### ขั้นตอน:
1. สร้าง **session key** แบบสุ่ม (AES 256-bit)
2. ใช้ session key เข้ารหัสข้อมูลจริง (symmetric encryption)
3. ใช้ public key ของผู้ใช้แต่ละคนเข้ารหัส session key (asymmetric encryption)
4. เก็บทั้ง encrypted data และ encrypted session keys
5. การถอดรหัส: ใช้ private key ถอด session key → ใช้ session key ถอดข้อมูลจริง

### ข้อดี:
- เข้ารหัสข้อมูลครั้งเดียว แต่หลายคนถอดได้
- เพิ่ม/ลบผู้ใช้ได้โดยไม่ต้องเข้ารหัสข้อมูลใหม่
- มีประสิทธิภาพกับข้อมูลขนาดใหญ่

## เตรียมสภาพแวดล้อม
- รัน `docker-compose up -d postgres` เพื่อให้บริการ Postgres พร้อมใช้งาน
- ยืนยันว่า pgcrypto extension ถูกเปิดใช้งานแล้ว (ดูที่ `docker/initdb/01-pgcrypto.sql`)
- เปิดใช้งาน virtualenv หรือติดตั้ง dependencies จาก `requirements.txt`

In [1]:
# ติดตั้ง dependencies ถ้าจำเป็น
# %pip install -q "psycopg[binary]>=3.1" SQLAlchemy>=2.0 pandas cryptography

## Import และตั้งค่าเชื่อมต่อฐานข้อมูล

In [2]:
import os
import secrets
import base64
from typing import Dict, List, Optional

import pandas as pd
from sqlalchemy import create_engine, Text, LargeBinary, Integer, select, text, and_
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column, relationship
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

# ค่าการเชื่อมต่อฐานข้อมูล
PG_HOST = os.getenv("PG_HOST", "localhost")
PG_PORT = int(os.getenv("PG_PORT", "5432"))
PG_DB = os.getenv("PG_DB", "encdb")
PG_USER = os.getenv("PG_USER", "encuser")
PG_PWD = os.getenv("PG_PASSWORD", "encpass")

engine = create_engine(
    f"postgresql+psycopg://{PG_USER}:{PG_PWD}@{PG_HOST}:{PG_PORT}/{PG_DB}",
    future=True,
)

print(f"เชื่อมต่อกับ PostgreSQL: {PG_HOST}:{PG_PORT}/{PG_DB}")

เชื่อมต่อกับ PostgreSQL: localhost:5432/encdb


## สร้างโมเดลสำหรับ Hybrid Encryption

เราจะสร้าง 3 ตาราง:
1. **users** - เก็บข้อมูลผู้ใช้และ public/private keys
2. **sensitive_documents** - เก็บข้อมูลที่เข้ารหัสด้วย session key
3. **document_access** - เก็บ encrypted session keys สำหรับแต่ละผู้ใช้ที่มีสิทธิ์

In [3]:
from sqlalchemy import ForeignKey

class Base(DeclarativeBase):
    pass


class User(Base):
    """เก็บข้อมูลผู้ใช้และ RSA key pairs"""
    __tablename__ = "users"
    
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    username: Mapped[str] = mapped_column(Text, unique=True, nullable=False)
    public_key: Mapped[bytes] = mapped_column(LargeBinary, nullable=False)  # PGP public key
    private_key: Mapped[bytes] = mapped_column(LargeBinary, nullable=False)  # PGP private key (encrypted)
    
    # Relationship
    document_accesses: Mapped[List["DocumentAccess"]] = relationship(back_populates="user")
    
    def __repr__(self) -> str:
        return f"<User id={self.id} username={self.username!r}>"


class SensitiveDocument(Base):
    """เก็บข้อมูลที่เข้ารหัสด้วย session key"""
    __tablename__ = "sensitive_documents"
    
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    title: Mapped[str] = mapped_column(Text, nullable=False)
    encrypted_content: Mapped[bytes] = mapped_column(LargeBinary, nullable=False)  # เข้ารหัสด้วย session key
    
    # Relationship
    document_accesses: Mapped[List["DocumentAccess"]] = relationship(back_populates="document")
    
    def __repr__(self) -> str:
        return f"<SensitiveDocument id={self.id} title={self.title!r}>"


class DocumentAccess(Base):
    """เก็บ session key ที่เข้ารหัสด้วย public key ของแต่ละผู้ใช้"""
    __tablename__ = "document_access"
    
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    document_id: Mapped[int] = mapped_column(Integer, ForeignKey("sensitive_documents.id"), nullable=False)
    user_id: Mapped[int] = mapped_column(Integer, ForeignKey("users.id"), nullable=False)
    encrypted_session_key: Mapped[bytes] = mapped_column(LargeBinary, nullable=False)  # session key เข้ารหัสด้วย user's public key
    
    # Relationships
    user: Mapped["User"] = relationship(back_populates="document_accesses")
    document: Mapped["SensitiveDocument"] = relationship(back_populates="document_accesses")
    
    def __repr__(self) -> str:
        return f"<DocumentAccess doc_id={self.document_id} user_id={self.user_id}>"


# สร้างตารางใหม่
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS document_access CASCADE"))
    conn.execute(text("DROP TABLE IF EXISTS sensitive_documents CASCADE"))
    conn.execute(text("DROP TABLE IF EXISTS users CASCADE"))

Base.metadata.create_all(engine)
print("สร้างตาราง users, sensitive_documents, document_access แล้ว")

สร้างตาราง users, sensitive_documents, document_access แล้ว


## สร้าง Helper Functions สำหรับ Hybrid Encryption

In [4]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

def generate_rsa_keypair(key_size: int = 2048) -> tuple[bytes, bytes]:
    """สร้าง RSA key pair และแปลงเป็น PEM format"""
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=key_size,
    )
    public_key = private_key.public_key()
    
    # แปลงเป็น PEM format
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )
    
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    return public_pem, private_pem


def encrypt_with_public_key(plaintext: str, public_key_pem: bytes) -> bytes:
    """เข้ารหัสข้อความด้วย RSA public key"""
    public_key = serialization.load_pem_public_key(public_key_pem)
    
    ciphertext = public_key.encrypt(
        plaintext.encode('utf-8'),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext


def decrypt_with_private_key(ciphertext: bytes, private_key_pem: bytes) -> str:
    """ถอดรหัสข้อความด้วย RSA private key"""
    private_key = serialization.load_pem_private_key(
        private_key_pem,
        password=None
    )
    
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext.decode('utf-8')


def generate_session_key() -> str:
    """สร้าง session key แบบสุ่ม (256-bit)"""
    # สร้าง 32 bytes (256 bits) random key
    return secrets.token_hex(32)


def create_user_with_keys(session: Session, username: str) -> User:
    """สร้างผู้ใช้พร้อม RSA key pair"""
    public_key, private_key = generate_rsa_keypair()
    user = User(
        username=username,
        public_key=public_key,
        private_key=private_key
    )
    session.add(user)
    session.flush()  # เพื่อให้ได้ user.id
    return user


print("Helper functions พร้อมใช้งาน")

Helper functions พร้อมใช้งาน


## สร้างผู้ใช้ทดสอบ
สร้างผู้ใช้ 3 คน: Alice, Bob, Charlie

In [5]:
with Session(engine) as session:
    alice = create_user_with_keys(session, "alice")
    bob = create_user_with_keys(session, "bob")
    charlie = create_user_with_keys(session, "charlie")
    session.commit()
    
    print(f"สร้างผู้ใช้:")
    print(f"  - Alice (ID: {alice.id})")
    print(f"  - Bob (ID: {bob.id})")
    print(f"  - Charlie (ID: {charlie.id})")

# แสดงข้อมูลผู้ใช้
with Session(engine) as session:
    users = session.execute(select(User)).scalars().all()
    df = pd.DataFrame([
        {
            "id": u.id,
            "username": u.username,
            "public_key_size": len(u.public_key),
            "private_key_size": len(u.private_key)
        } for u in users
    ])
    display(df)

สร้างผู้ใช้:
  - Alice (ID: 1)
  - Bob (ID: 2)
  - Charlie (ID: 3)


,id,username,public_key_size,private_key_size
0,1,alice,451,1675
1,2,bob,451,1675
2,3,charlie,451,1675


## สร้างเอกสารและเข้ารหัสด้วย Hybrid Encryption

### ขั้นตอน:
1. สร้าง session key แบบสุ่ม (256-bit)
2. ใช้ `pgp_sym_encrypt()` (pgcrypto) เข้ารหัสเนื้อหาด้วย session key
3. ใช้ RSA public key encryption (Python cryptography) เข้ารหัส session key สำหรับแต่ละผู้ใช้
4. บันทึกทั้ง encrypted content และ encrypted session keys

**หมายเหตุ:** 
- ใช้ pgcrypto สำหรับ symmetric encryption (AES) - เร็วและเหมาะกับข้อมูลขนาดใหญ่
- ใช้ Python cryptography สำหรับ asymmetric encryption (RSA) - เพราะ pgcrypto ต้องการ OpenPGP format

In [6]:
def create_encrypted_document(
    session: Session,
    title: str,
    content: str,
    authorized_user_ids: List[int]
) -> SensitiveDocument:
    """
    สร้างเอกสารที่เข้ารหัสและให้สิทธิ์เข้าถึงแก่ผู้ใช้ที่ระบุ
    
    Args:
        session: SQLAlchemy session
        title: ชื่อเอกสาร
        content: เนื้อหาที่ต้องการเข้ารหัส
        authorized_user_ids: รายการ user IDs ที่มีสิทธิ์เข้าถึง
    """
    # 1. สร้าง session key
    session_key = generate_session_key()
    
    # 2. เข้ารหัสเนื้อหาด้วย session key (symmetric) ใช้ pgcrypto
    encrypted_content = session.execute(
        text("SELECT pgp_sym_encrypt(:content, :key)"),
        {"content": content, "key": session_key}
    ).scalar()
    
    # 3. สร้างเอกสาร
    doc = SensitiveDocument(
        title=title,
        encrypted_content=encrypted_content
    )
    session.add(doc)
    session.flush()
    
    # 4. เข้ารหัส session key ด้วย public key ของแต่ละผู้ใช้ (asymmetric) ใช้ Python
    for user_id in authorized_user_ids:
        user = session.get(User, user_id)
        if not user:
            print(f"⚠️  ไม่พบผู้ใช้ ID {user_id}")
            continue
        
        # เข้ารหัส session key ด้วย public key ของผู้ใช้ (ใช้ cryptography library)
        encrypted_session_key = encrypt_with_public_key(session_key, user.public_key)
        
        # บันทึก encrypted session key
        access = DocumentAccess(
            document_id=doc.id,
            user_id=user.id,
            encrypted_session_key=encrypted_session_key
        )
        session.add(access)
    
    session.flush()
    return doc


# สร้างเอกสารทดสอบ
with Session(engine) as session:
    # เอกสารที่ Alice และ Bob เข้าถึงได้
    doc1 = create_encrypted_document(
        session,
        title="Project Alpha Budget",
        content="งบประมาณโครงการ Alpha: 5,000,000 บาท\nรายละเอียดค่าใช้จ่าย...",
        authorized_user_ids=[1, 2]  # Alice และ Bob
    )
    
    # เอกสารที่ทั้งสามคนเข้าถึงได้
    doc2 = create_encrypted_document(
        session,
        title="Company Confidential Data",
        content="ข้อมูลความลับของบริษัท\nเลขบัญชีธนาคาร: 123-456-7890\nรหัสผ่าน: SecretP@ss",
        authorized_user_ids=[1, 2, 3]  # Alice, Bob, และ Charlie
    )
    
    session.commit()
    print(f"✅ สร้างเอกสาร '{doc1.title}' (ID: {doc1.id}) - อนุญาตให้ Alice และ Bob")
    print(f"✅ สร้างเอกสาร '{doc2.title}' (ID: {doc2.id}) - อนุญาตให้ Alice, Bob และ Charlie")

✅ สร้างเอกสาร 'Project Alpha Budget' (ID: 1) - อนุญาตให้ Alice และ Bob
✅ สร้างเอกสาร 'Company Confidential Data' (ID: 2) - อนุญาตให้ Alice, Bob และ Charlie


## ตรวจสอบข้อมูลที่เก็บในฐานข้อมูล

ดูว่าข้อมูลถูกเข้ารหัสอย่างไร

In [7]:
# ดูเอกสารที่เข้ารหัส
with engine.begin() as conn:
    docs = conn.execute(
        text("""
            SELECT id, title, 
                   length(encrypted_content) as encrypted_size,
                   encode(substring(encrypted_content from 1 for 40), 'hex') as preview
            FROM sensitive_documents
            ORDER BY id
        """)
    ).fetchall()

print("📄 เอกสารที่เข้ารหัส:")
display(pd.DataFrame(docs, columns=["id", "title", "encrypted_size", "preview (hex)"]))

📄 เอกสารที่เข้ารหัส:


,id,title,encrypted_size,preview (hex)
0,1,Project Alpha Budget,202,c30d04070302ccb6d579d907ab997ed2b901c0cdd5c43f...
1,2,Company Confidential Data,227,c30d040703020466df3e5032ad576ad2c011010ae1d9dd...


In [8]:
# ดู encrypted session keys
with engine.begin() as conn:
    accesses = conn.execute(
        text("""
            SELECT 
                da.id,
                sd.title as document,
                u.username,
                length(da.encrypted_session_key) as key_size
            FROM document_access da
            JOIN sensitive_documents sd ON da.document_id = sd.id
            JOIN users u ON da.user_id = u.id
            ORDER BY da.document_id, u.username
        """)
    ).fetchall()

print("\n🔑 Encrypted Session Keys (ใครมีสิทธิ์เข้าถึงเอกสารอะไร):")
display(pd.DataFrame(accesses, columns=["id", "document", "username", "encrypted_key_size"]))


🔑 Encrypted Session Keys (ใครมีสิทธิ์เข้าถึงเอกสารอะไร):


,id,document,username,encrypted_key_size
0,1,Project Alpha Budget,alice,256
1,2,Project Alpha Budget,bob,256
2,3,Company Confidential Data,alice,256
3,4,Company Confidential Data,bob,256
4,5,Company Confidential Data,charlie,256


## ถอดรหัสเอกสารโดยผู้ใช้

### ขั้นตอนการถอดรหัส:
1. ดึง encrypted session key ของผู้ใช้จากตาราง `document_access`
2. ใช้ RSA private key (Python cryptography) ถอดรหัส session key
3. ใช้ `pgp_sym_decrypt()` (pgcrypto) กับ session key เพื่อถอดเนื้อหา

In [10]:
def decrypt_document_for_user(
    session: Session,
    document_id: int,
    user_id: int
) -> Optional[str]:
    """
    ถอดรหัสเอกสารสำหรับผู้ใช้ที่ระบุ
    
    Args:
        session: SQLAlchemy session
        document_id: ID ของเอกสาร
        user_id: ID ของผู้ใช้
        
    Returns:
        เนื้อหาที่ถอดรหัสแล้ว หรือ None ถ้าไม่มีสิทธิ์
    """
    # ตรวจสอบสิทธิ์
    access = session.execute(
        select(DocumentAccess)
        .where(and_(
            DocumentAccess.document_id == document_id,
            DocumentAccess.user_id == user_id
        ))
    ).scalar_one_or_none()
    
    if not access:
        return None
    
    # ดึงข้อมูล
    user = session.get(User, user_id)
    doc = session.get(SensitiveDocument, document_id)
    
    # 1. ถอด session key ด้วย private key (ใช้ cryptography library)
    session_key = decrypt_with_private_key(access.encrypted_session_key, user.private_key)
    
    # 2. ถอดเนื้อหาด้วย session key (ใช้ pgcrypto)
    content = session.execute(
        text("SELECT pgp_sym_decrypt(:encrypted_content, :session_key)"),
        {
            "encrypted_content": doc.encrypted_content,
            "session_key": session_key
        }
    ).scalar()
    
    # pgp_sym_decrypt คืนค่าเป็น string แล้ว (ใน psycopg3)
    return content


# ทดสอบถอดรหัส
print("🔓 ทดสอบการถอดรหัส:\n")

with Session(engine) as session:
    # Alice อ่านเอกสาร 1
    content = decrypt_document_for_user(session, document_id=1, user_id=1)
    print("Alice อ่านเอกสาร 'Project Alpha Budget':")
    print(f"  {content}\n" if content else "  ❌ ไม่มีสิทธิ์\n")
    
    # Bob อ่านเอกสาร 1
    content = decrypt_document_for_user(session, document_id=1, user_id=2)
    print("Bob อ่านเอกสาร 'Project Alpha Budget':")
    print(f"  {content}\n" if content else "  ❌ ไม่มีสิทธิ์\n")
    
    # Charlie พยายามอ่านเอกสาร 1 (ไม่มีสิทธิ์)
    content = decrypt_document_for_user(session, document_id=1, user_id=3)
    print("Charlie อ่านเอกสาร 'Project Alpha Budget':")
    print(f"  {content}\n" if content else "  ❌ ไม่มีสิทธิ์\n")
    
    # Charlie อ่านเอกสาร 2 (มีสิทธิ์)
    content = decrypt_document_for_user(session, document_id=2, user_id=3)
    print("Charlie อ่านเอกสาร 'Company Confidential Data':")
    print(f"  {content}\n" if content else "  ❌ ไม่มีสิทธิ์\n")

🔓 ทดสอบการถอดรหัส:

Alice อ่านเอกสาร 'Project Alpha Budget':
  งบประมาณโครงการ Alpha: 5,000,000 บาท
รายละเอียดค่าใช้จ่าย...

Bob อ่านเอกสาร 'Project Alpha Budget':
  งบประมาณโครงการ Alpha: 5,000,000 บาท
รายละเอียดค่าใช้จ่าย...

Charlie อ่านเอกสาร 'Project Alpha Budget':
  ❌ ไม่มีสิทธิ์

Charlie อ่านเอกสาร 'Company Confidential Data':
  ข้อมูลความลับของบริษัท
เลขบัญชีธนาคาร: 123-456-7890
รหัสผ่าน: SecretP@ss

Charlie อ่านเอกสาร 'Company Confidential Data':
  ข้อมูลความลับของบริษัท
เลขบัญชีธนาคาร: 123-456-7890
รหัสผ่าน: SecretP@ss



## เพิ่มสิทธิ์ผู้ใช้ใหม่ (Grant Access)

เมื่อต้องการให้ผู้ใช้คนใหม่เข้าถึงเอกสารที่มีอยู่:
1. ผู้ใช้ที่มีสิทธิ์อยู่แล้วถอด session key ออกมา
2. เข้ารหัส session key นั้นด้วย public key ของผู้ใช้ใหม่
3. บันทึก encrypted session key ใหม่ลง `document_access`

**ข้อดี:** ไม่ต้องเข้ารหัสข้อมูลใหม่ทั้งหมด

In [11]:
def grant_document_access(
    session: Session,
    document_id: int,
    grantor_user_id: int,
    new_user_id: int
) -> bool:
    """
    ให้สิทธิ์ผู้ใช้ใหม่ในการเข้าถึงเอกสาร
    
    Args:
        session: SQLAlchemy session
        document_id: ID ของเอกสาร
        grantor_user_id: ID ของผู้ใช้ที่มีสิทธิ์อยู่แล้ว (คนให้สิทธิ์)
        new_user_id: ID ของผู้ใช้ใหม่ที่จะได้รับสิทธิ์
        
    Returns:
        True ถ้าสำเร็จ, False ถ้าไม่สำเร็จ
    """
    # ตรวจสอบว่าผู้ให้สิทธิ์มีสิทธิ์อยู่หรือไม่
    grantor_access = session.execute(
        select(DocumentAccess)
        .where(and_(
            DocumentAccess.document_id == document_id,
            DocumentAccess.user_id == grantor_user_id
        ))
    ).scalar_one_or_none()
    
    if not grantor_access:
        print(f"❌ ผู้ใช้ ID {grantor_user_id} ไม่มีสิทธิ์เข้าถึงเอกสาร ID {document_id}")
        return False
    
    # ตรวจสอบว่าผู้ใช้ใหม่มีสิทธิ์อยู่แล้วหรือไม่
    existing_access = session.execute(
        select(DocumentAccess)
        .where(and_(
            DocumentAccess.document_id == document_id,
            DocumentAccess.user_id == new_user_id
        ))
    ).scalar_one_or_none()
    
    if existing_access:
        print(f"⚠️  ผู้ใช้ ID {new_user_id} มีสิทธิ์อยู่แล้ว")
        return False
    
    # ดึงข้อมูล
    grantor = session.get(User, grantor_user_id)
    new_user = session.get(User, new_user_id)
    
    if not new_user:
        print(f"❌ ไม่พบผู้ใช้ ID {new_user_id}")
        return False
    
    # 1. ถอด session key ด้วย private key ของผู้ให้สิทธิ์ (ใช้ cryptography library)
    session_key = decrypt_with_private_key(grantor_access.encrypted_session_key, grantor.private_key)
    
    # 2. เข้ารหัส session key ด้วย public key ของผู้ใช้ใหม่ (ใช้ cryptography library)
    encrypted_session_key = encrypt_with_public_key(session_key, new_user.public_key)
    
    # 3. บันทึกสิทธิ์ใหม่
    new_access = DocumentAccess(
        document_id=document_id,
        user_id=new_user_id,
        encrypted_session_key=encrypted_session_key
    )
    session.add(new_access)
    session.flush()
    
    print(f"✅ เพิ่มสิทธิ์ให้ {new_user.username} เข้าถึงเอกสาร ID {document_id} แล้ว")
    return True


# ทดสอบ: Alice ให้สิทธิ์ Charlie เข้าถึงเอกสาร 1
print("📋 เพิ่มสิทธิ์ผู้ใช้ใหม่:\n")
with Session(engine) as session:
    # ก่อนให้สิทธิ์ - Charlie ไม่สามารถอ่านเอกสาร 1
    content = decrypt_document_for_user(session, document_id=1, user_id=3)
    print("Charlie อ่านเอกสาร 1 (ก่อนได้รับสิทธิ์):")
    print(f"  {content if content else '❌ ไม่มีสิทธิ์'}\n")
    
    # Alice ให้สิทธิ์ Charlie
    grant_document_access(session, document_id=1, grantor_user_id=1, new_user_id=3)
    session.commit()
    print()
    
    # หลังให้สิทธิ์ - Charlie สามารถอ่านได้
    content = decrypt_document_for_user(session, document_id=1, user_id=3)
    print("Charlie อ่านเอกสาร 1 (หลังได้รับสิทธิ์):")
    print(f"  {content if content else '❌ ไม่มีสิทธิ์'}")

📋 เพิ่มสิทธิ์ผู้ใช้ใหม่:

Charlie อ่านเอกสาร 1 (ก่อนได้รับสิทธิ์):
  ❌ ไม่มีสิทธิ์

✅ เพิ่มสิทธิ์ให้ charlie เข้าถึงเอกสาร ID 1 แล้ว

Charlie อ่านเอกสาร 1 (หลังได้รับสิทธิ์):
  งบประมาณโครงการ Alpha: 5,000,000 บาท
รายละเอียดค่าใช้จ่าย...


## เพิกถอนสิทธิ์ผู้ใช้ (Revoke Access)

เมื่อต้องการเพิกถอนสิทธิ์ผู้ใช้:
- ลบ record ใน `document_access` ที่เกี่ยวข้อง
- ผู้ใช้จะไม่สามารถถอดรหัสได้อีก (เพราะไม่มี encrypted session key)

**ข้อควรระวัง:** ถ้าผู้ใช้เคย cache session key ไว้ ก็ยังถอดได้ — ต้องมีมาตรการเพิ่มเติมในระบบจริง

In [12]:
def revoke_document_access(
    session: Session,
    document_id: int,
    user_id: int
) -> bool:
    """
    เพิกถอนสิทธิ์ผู้ใช้ในการเข้าถึงเอกสาร
    
    Args:
        session: SQLAlchemy session
        document_id: ID ของเอกสาร
        user_id: ID ของผู้ใช้ที่จะถูกเพิกถอนสิทธิ์
        
    Returns:
        True ถ้าสำเร็จ, False ถ้าไม่พบสิทธิ์
    """
    access = session.execute(
        select(DocumentAccess)
        .where(and_(
            DocumentAccess.document_id == document_id,
            DocumentAccess.user_id == user_id
        ))
    ).scalar_one_or_none()
    
    if not access:
        print(f"⚠️  ไม่พบสิทธิ์สำหรับผู้ใช้ ID {user_id} กับเอกสาร ID {document_id}")
        return False
    
    user = session.get(User, user_id)
    doc = session.get(SensitiveDocument, document_id)
    
    session.delete(access)
    session.flush()
    
    print(f"✅ เพิกถอนสิทธิ์ {user.username} จากเอกสาร '{doc.title}' แล้ว")
    return True


# ทดสอบ: เพิกถอนสิทธิ์ Bob จากเอกสาร 1
print("🚫 เพิกถอนสิทธิ์ผู้ใช้:\n")
with Session(engine) as session:
    # ก่อนเพิกถอน - Bob สามารถอ่านได้
    content = decrypt_document_for_user(session, document_id=1, user_id=2)
    print("Bob อ่านเอกสาร 1 (ก่อนถูกเพิกถอนสิทธิ์):")
    print(f"  {'✅ อ่านได้' if content else '❌ ไม่มีสิทธิ์'}\n")
    
    # เพิกถอนสิทธิ์ Bob
    revoke_document_access(session, document_id=1, user_id=2)
    session.commit()
    print()
    
    # หลังเพิกถอน - Bob ไม่สามารถอ่านได้
    content = decrypt_document_for_user(session, document_id=1, user_id=2)
    print("Bob อ่านเอกสาร 1 (หลังถูกเพิกถอนสิทธิ์):")
    print(f"  {'✅ อ่านได้' if content else '❌ ไม่มีสิทธิ์'}")

🚫 เพิกถอนสิทธิ์ผู้ใช้:

Bob อ่านเอกสาร 1 (ก่อนถูกเพิกถอนสิทธิ์):
  ✅ อ่านได้

✅ เพิกถอนสิทธิ์ bob จากเอกสาร 'Project Alpha Budget' แล้ว

Bob อ่านเอกสาร 1 (หลังถูกเพิกถอนสิทธิ์):
  ❌ ไม่มีสิทธิ์


## สรุปสิทธิ์การเข้าถึงปัจจุบัน

In [13]:
with engine.begin() as conn:
    summary = conn.execute(
        text("""
            SELECT 
                sd.id as doc_id,
                sd.title,
                string_agg(u.username, ', ' ORDER BY u.username) as authorized_users
            FROM sensitive_documents sd
            LEFT JOIN document_access da ON sd.id = da.document_id
            LEFT JOIN users u ON da.user_id = u.id
            GROUP BY sd.id, sd.title
            ORDER BY sd.id
        """)
    ).fetchall()

print("\n📊 สรุปสิทธิ์การเข้าถึงปัจจุบัน:")
display(pd.DataFrame(summary, columns=["doc_id", "title", "authorized_users"]))


📊 สรุปสิทธิ์การเข้าถึงปัจจุบัน:


,doc_id,title,authorized_users
0,1,Project Alpha Budget,"alice, charlie"
1,2,Company Confidential Data,"alice, bob, charlie"


## สรุปและข้อควรพิจารณา

### ข้อดีของ Hybrid Encryption:
1. ✅ **ประสิทธิภาพ** - เข้ารหัสข้อมูลครั้งเดียว แม้มีผู้ใช้หลายคน
2. ✅ **ยืดหยุ่น** - เพิ่ม/ลบผู้ใช้ได้โดยไม่ต้องเข้ารหัสข้อมูลใหม่
3. ✅ **ปลอดภัย** - ใช้ AES สำหรับข้อมูล (เร็ว) และ RSA สำหรับ key (ปลอดภัย)
4. ✅ **Scalable** - เหมาะกับข้อมูลขนาดใหญ่และผู้ใช้จำนวนมาก

### ข้อควรระวัง:
1. ⚠️ **Key Management** - ต้องจัดการ private keys อย่างปลอดภัย
2. ⚠️ **Session Key Rotation** - ควรหมุนเวียน session key เป็นระยะ (ต้องเข้ารหัสข้อมูลใหม่)
3. ⚠️ **Caching** - ถ้า cache session key ไว้ การเพิกถอนสิทธิ์อาจไม่ทันที
4. ⚠️ **Backup** - ต้องมีวิธี backup และ recover private keys

### การใช้งานจริง:
- ควรเก็บ private keys แบบเข้ารหัส (password-protected)
- ใช้ HSM หรือ Key Management Service สำหรับ keys ที่สำคัญ
- พิจารณา audit log สำหรับการเข้าถึงข้อมูล
- ใช้ระบบ key rotation อัตโนมัติ